<a href="https://colab.research.google.com/github/Nutsudapenpong/AIprototype2022/blob/main/MyownDensenet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd /content/drive/MyDrive/New_yolo/Densenet

/content/drive/MyDrive/New_yolo/Densenet


In [4]:
!wget http://www.vision.caltech.edu/Image_Datasets/Caltech256/256_ObjectCategories.tar

--2022-05-29 15:38:57--  http://www.vision.caltech.edu/Image_Datasets/Caltech256/256_ObjectCategories.tar
Resolving www.vision.caltech.edu (www.vision.caltech.edu)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to www.vision.caltech.edu (www.vision.caltech.edu)|185.199.108.153|:80... connected.
HTTP request sent, awaiting response... 404 Not Found
2022-05-29 15:38:57 ERROR 404: Not Found.



In [5]:
#a library to simplify training
!pip install pytorch-ignite

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 259 kB 7.3 MB/s 


In [ ]:
#for unzipping tar files
!tar -xf 256_ObjectCategories.tar

In [ ]:
import torch
import torchvision
from torchvision import datasets, transforms #For loading our dataset
from torch import nn, optim # NN module and optimizers
from torch.nn import functional as F # Certain function, like sigmoid and tanh
from torch.utils.data import DataLoader, sampler, random_split
from torchvision import models # This is where we get our model.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
#For retina displays only

In [ ]:
import numpy as np

In [ ]:
from ignite.engine import Engine, Events, create_supervised_trainer, create_supervised_evaluator
from ignite.handlers import ModelCheckpoint, EarlyStopping
from ignite.metrics import Accuracy, Loss, RunningAverage, ConfusionMatrix

In [ ]:
#writing a helper function to load data in
def get_data_loaders(data_dir, batch_size):
  transform = transform.Compose([transforms.Resize(255), 
                                 transforms.CenterCrop(224),
                                 transforms.ToTensor()])
  #Changing the image so that we can use it in our model
  all_images = datasets.ImageFolder(data_dir, transform=transform)
  train_images_len = int(len(all_images) * 0.75) #75% of images will be trained
  valid_images_len = int(len(all_images) - train_images_len)/2)
  test_images_len = int((len(all_images) - train_images_len - valid_images_len))
  train_data, val_data, test_data = random_split(all_images, [train_images_len, val_images_len, test_images_len])
  train_loader = DataLoader(train_data, batch_size=batch_size)
  test_loader = DataLoader(test_data, batch_size=batch_size)
  val_loader = DataLoader(val_data, batch_size=batch_size)
  return (train_loader, val_loader, test_loader), all_images.classes

In [ ]:
(train_loader, val_loader, test_loader), classes = get_data_loader("/content/256_ObjectCategories", 64)

In [ ]:
print(classes)

In [ ]:
#build a helper function to format the text
#Goal is to turn a string like 007.bat to bat
import re
def formatText(string):
  string = string[4:]
  return string.replace("_", " ")
print(formatText(classes[0]))

In [ ]:
#Display training images
dataiter = iter(train_loader)
images, labels = dataiter.next()

fig=plt.figure(figure=(25,4))
for idx in np.arange(20): #only 20 images
  ax = fig.add_subplot(2, 20/2, idx + 1, xticks=[], yticks=[])
  #We don't need axes for the images
  plt.imshow(np.transpose(images[idx], (1, 2, 0)))
  ax.set_title(formatText(classes[labels[idx]]))

#training

In [ ]:
#loading model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

In [ ]:
model = models.densenet201(pretrained=True) #using densenet 201

In [ ]:
print(model)

In [ ]:
#We need to change the last layer to have a output shape of 257
print(model.classifier.in_features)
print(model.classifier.out_features)

In [ ]:
for param in model.parameters():
  param.requires_grad = False #We are freezing the model since it was trained on imageet and we do not need to train that part agian 

In [ ]:
n_inputs = model.classifier.in_faetures
last_layer nn.Linear(n_inputs, len(classes))
model.classifier = last_layer
print(model.classifier.out_features)

In [ ]:
#Moving our model to GPU
model = model.to(device)

In [ ]:
#Defining our optimizer and loss function
criterion = nn.CrossEntropyLoss() #This is our loss function
optimizer = optim.Adam(model.classifier.parameters()) #Feed parameters to the model

In [ ]:
#Training model
training_history = {'accuracy':[], 'loss':[]}
validaton_history = {'accuracy':[], 'loss':[]}

In [ ]:
#Now we will use PyTorch Ignite's feaures
trainer = create_supervised_trainer(model, optimizer, criterion, device=device)
evaluator = creae_supervised_evaluator(model,
                                       device=device
                                       metrics={
                                           'accuracy':Accuracy(),
                                           'Loss': Loss(criterion),
                                           'cm':ConfusionMatrix(len(classes))
                                      ( })
#Creating even handlers to show our training progress
@trainer.on(Events.ITERATION_COMPLETED)
def log_a_dot(engine):
  print(".", end="")

@trainer.on(Events.EPOCH_COMPLETED)
def log_training_results(trainer):
  evaluator.run(train_loader)
  metrics = evaluator.state.metrics
  accuracy = metrics['accuracy']*100
  loss = metrics['loss']
  training_history['accuracy'].append(accuracy)
  training_history['loss'].append(loss)
  print()
  print(f"Training results - Epoch: {trainer.state.epoch} Avg accuracy: {accuracy} Loss: {loss}")

@trainer.on(Events.EPOCH_COMPLETED)
def log_validation_results(trainer):
  evaluator.run(val_loader)
  metrics = evaluator.state.metrics
  accuracy = metrics['accuracy']*100
  loss = metrics['loss']
  validation_history['accuracy'].append(accuracy)
  validaion_history['loss'].append(loss)
  print()
  print(f"Validation results - Epoch: {trainer.state.epoch} Avg accuracy: {accuracy} Loss: {loss}")

In [ ]:
trainer.run(train_loader, max_epochs=5)